## ***ImportFinancialData***

Code will show how it let user set filter and get dataframes for each statements which containts all companies sector.


Steps:
1. select specific sector you're interested in - USER
2. scrape sectors company from Barron's website
3. scrape financial data from MarketWatch website
3. get corporate table (income, balance sheet, cash flow, ratio table) (with target currency specified)

<u>*Note*</u>: about **ratio table** it is custom and indices will have objective to indentify main characteristics that show how corporate is going viewing itself and comparing with others*

## Data extraction

**<u>SetUp</u>**

In [1]:
# pip install --upgrade pandas pyarrow

In [2]:
# pip install google-auth

In [1]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from io import StringIO
import time
import numpy as np
import threading

In [2]:
# Make sure pandas version is over or same of 2.1.0 
print(pd.__version__)

2.1.0


In [3]:
# setting 'None' to see entire table as defoult output
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format

**<U>Import libraries UDF**

In [4]:
import sys
sys.path.append('C:/Users/utente/Desktop/Master ACF/financial_modeling/dataset/notebook/UDF/finance_udf')
import Download_Sectors, Get_Corporates_Analysis_Data, Get_Financial_Analysis_Data, country_codes_corporate, Visual_Corporate

**<u>Read *countries_and_currencies* file**

In [5]:
from country_codes_corporate import countries_and_currencies

In [6]:
countries_and_currencies = pd.DataFrame(countries_and_currencies)

In [7]:
len(countries_and_currencies)

189

**<u>Sectors available**

In [11]:
# --- 1. RUN THIS IF YOU DIDN'DOWNLOAD YET
# Download_Sectors.Download_Sectors()

In [8]:
# --- 2. RUN THIS IF YOU'VE ALREADY DOWNLOADED AND HAVE EXCEL FILE IN FOLDER
link_file = '1jpK3Y2Xt5c1V_fiIsgO9C6mg5pYYju0G' # available to anyone with the link

# Read file on google drive
url=f'https://drive.google.com/file/d/{link_file}/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
path_sector_file = pd.read_excel(url)

In [9]:
# Sectors available
path_sector_file['sector'].unique()

array(['Business/Consumer Services', 'Industrial Goods',
       'Transportation/Logistics', 'Consumer Goods', 'Energy',
       'Basic Materials/Resources', 'Automotive', 'Financial Services',
       'Health Care/Life Sciences', 'Media/Entertainment',
       'Real Estate/Construction', 'Retail/Wholesale', 'Technology',
       'Utilities', 'Agriculture', 'Leisure/Arts/Hospitality',
       'Telecommunication Services'], dtype=object)

### User required settings

**<u>1. Filter: regions and currency** (EU and EEU) --> select countries needs

In [10]:
eu_regions = ['Austria', 'Belgium','Bulgaria', 'Croatia',
        'Cyprus', 'Czechia', 'Denmark', 'Estonia',
        'Finland', 'France', 'Germany', 'Greece',
        'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania',
        'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal',
        'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden',
        'Iceland', 'Norway']

regions = eu_regions

**<u>2. Specify target currency to conversion**

In [11]:
# Specify the target currency, in which all the different currencies will be converted
target_currency = 'USD'

**<u>3. Specify Sector**

In [12]:
sectors = ['Consumer Goods']

**<u>Optional filter**: get just for specific letter/specific

In [13]:
alp = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

**NOTE**: following outputs
* Financial table
* Statements (BS, IS, CF, RT)

## <u>Financial table

Indices will get:
* Market Cap
* Market Cap (converted)
* Beta
* Average Volume
* Shares Outstanding
* P/E Ratio
* Yield(%)Public Float

**<u>Get financial data**

In [14]:
Financial_df = Get_Financial_Analysis_Data.finacial_analysis(sectors, eu_regions, alp, path_sector_file)
# Error below means the model found no source data for those companies

Error:  6 Adris Grupa d.d. Pfd.
Error:  6 Adris Grupa d.d.
Error:  32 Atlantic Grupa d.d.
Error:  10 Bermas S.A. Suceava
Error:  1 Cakovecki Mlinovi d.d.
Error:  6 Cateks d.d.
Error:  27 Conted S.A.
Error:  5 Electroarges S.A.
Error:  13 Granolio d.d.
Error:  17 Hydra Games S.A.
Error:  1 Jadran Tvornica Carapa d.d.
Error:  9 Koestlin d.d.
Error:  11 Kras d.d.
Error:  12 Kutjevo d.d.
Error:  12 Metaversum S.A.
Error:  14 Norse Impact AB
Error:  12 Podravka Prehrambena Industrija d.d.
Error:  19 Purcari Wineries PCL
Error:  6 Saponia d.d.
Error:  6 Tekstilpromet d.d.
Error:  4 Ves S.A.
Error:  5 Zordix AB Series B


**<u>Currency conversation**

In [15]:
Financial_df1, currencies_rates_df = Get_Financial_Analysis_Data.convert_to_target(Financial_df, target_currency, countries_and_currencies)

In [16]:
len(Financial_df1)

518

In [17]:
Financial_df1.head(5)

,Country,Sector,Exchange,Company Name,Ticker,Symbol,Market Cap,Market Cap (USD),Beta,Average Volume,Shares Outstanding,P/E Ratio,Yield(%),Public Float
0,Sweden,Food Products,XSTO,AAK AB,AAK,SEK,kr51760000.0,4633259.93,NaN,205.60,259560.00,21.85,1.36,175610.00
1,France,Sports Goods,XPAR,Abeo S.A.,ABEO,EUR,€125870.0,134232.70,NaN,481.23,7540.00,18.21,2.02,952.87
2,Netherlands,Food Products,XAMS,Acomo N.V.,ACOMO,EUR,€584540.0,623376.35,NaN,11.33,29580.00,12.96,6.00,18110.00
3,Germany,Food Products,XFRA,ADM Hamburg AG,OEL,EUR,€232130.0,247552.52,0.56,0.42,NaN,219.13,4.50,38.94
4,Hungary,Footwear,XBUD,adidas AG,ADID,HUF,Ft11200000000.0,31147770.31,0.85,1.17,180000.00,NaN,0.41,157500.00


## <u>Corporate data

**<u>4. Year setting**
will be taken the available years from the source, so if you have defined years not available you will not get errors.

In [18]:
Years = [2022,2021,2020,2019] # year statements refferred to

**Get BS, IS, CF data**

In [20]:
income_statement_annual_df,balance_sheet_annual_df,cash_flow_annual_df = Get_Corporates_Analysis_Data.Corporates_analysis(sectors, regions,Years, alp, path_sector_file)
# Error below means the model found no source data for those companies

error: 6 Adris Grupa d.d.
error: 6 Adris Grupa d.d. Pfd.
error: 27 Art Games Studio S.A.
error: 30 Asmodev S.A.
error: 32 Atlantic Grupa d.d.
error: 10 Bermas S.A. Suceava
error: 21 BTC Studios S.A.
error: 22 Bulgarska Roza AD
error: 1 Cabasse S.A.
error: 1 Cakovecki Mlinovi d.d.
error: 5 Carbon Studio S.A.
error: 6 Cateks d.d.
error: 27 Console Labs S.A.
error: 27 Conted S.A.
error: 5 Decotex AD
error: 12 Don't Nod Entertainment S.A.
error: 12 Drageus Games S.A.
error: 13 Duality S.A.
error: 4 Eevia Health Oyj
error: 5 Electroarges S.A.
error: 14 Eurosnack S.A.
error: 3 Fenix Outdoor International Ltd. Series B
error: 10 Franck Deville
error: 11 Frozen Way S.A.
error: 1 Galeo Concept
error: 2 Gamedust S.A.
error: 2 Games Box S.A.
error: 2 Gaming Factory S.A.
error: 2 Gamivo S.A.
error: 12 Gradus AD Stara Zagora
error: 13 Granolio d.d.
error: 15 Grolleau
error: 1 Hagen Bikes Holding A.S.
error: 3 Hannun S.A.
error: 10 HiProMine S.A.
error: 17 Huuuge Inc.
error: 17 Hval Sjokoladefabrikk

**Creating Ratio Table**

In [21]:
ratio_table_annual_df = Get_Corporates_Analysis_Data.get_ratio_table(income_statement_annual_df,balance_sheet_annual_df,cash_flow_annual_df)

**Formatting and modelling data**

In [22]:
income_statement_annual_df,balance_sheet_annual_df,cash_flow_annual_df = Get_Corporates_Analysis_Data.Corporates_Table_Formatting(income_statement_annual_df,balance_sheet_annual_df,cash_flow_annual_df)

**Currency conversation**

In [23]:
income_statement_annual_df1, balance_sheet_annual_df1, cash_flow_annual_df1 = Get_Corporates_Analysis_Data.convert_to_target(income_statement_annual_df,balance_sheet_annual_df,cash_flow_annual_df,Years,target_currency,countries_and_currencies)

## Mange potential duplicates found

**Duplicates definition:** rows which has same 'Sector','Super Sector','Industry' and 'Year' and values columns

**NOTE:** will not be removed equal companies which show values different currencies, because exchange rate are not exactly the same

**<u>Replacing year to int type**

In [24]:
income_statement_annual_df1['Year'] = income_statement_annual_df1['Year'].astype(int)
balance_sheet_annual_df1['Year'] = balance_sheet_annual_df1['Year'].astype(int)
cash_flow_annual_df1['Year'] = cash_flow_annual_df1['Year'].astype(int)
ratio_table_annual_df['Year'] = ratio_table_annual_df['Year'].astype(int)

**<u>Get duplicates</u>**

In [25]:
def get_duplicates(df):
    # Get column from year (included) over
    index_year = df.columns.get_loc('Year')
    columns_from_year = df.columns[index_year:].tolist()

    # Add 'Sector', 'Super Sector', 'industry_name' columns to check duplicates
    columns_to_check = columns_from_year + ['Sector', 'Super Sector', 'Industry']

    # Sort values
    sort_columns = ['Country', 'Year'] + columns_from_year
    sorted_df = df.sort_values(by=sort_columns, ascending=True)

    # Identify rows duplicated
    duplicates = sorted_df[sorted_df.duplicated(subset=columns_to_check, keep=False)]

    return duplicates.sort_values(by=['Company Name', 'Year'], ascending=True)

# Call udf
income_statement_duplicates = get_duplicates(income_statement_annual_df1)
balance_sheet_annual_duplicates = get_duplicates(balance_sheet_annual_df1)
cash_flow_annual_duplicates = get_duplicates(cash_flow_annual_df1)
ratio_table_duplicates = get_duplicates(ratio_table_annual_df)

In [ ]:
# income_statement_duplicates.sort_values(by=['Company Name','Year'], ascending=True)

In [26]:
# Duplicates number
len(cash_flow_annual_duplicates['Company Name'].unique())

37

**<u>Remove duplicated values</u>**

In [27]:
def remove_and_sort_duplicates(df):
    # Get columns from "Year" onwards
    index_year = df.columns.get_loc('Year')
    columns_from_year = df.columns[index_year:].tolist()

    # Sort values
    sort_columns = ['Country', 'Year'] + columns_from_year
    sorted_df = df.sort_values(by=sort_columns, ascending=True)

    # Remove duplicates and sort again
    other_col = ['Sector','Super Sector','industry_name']
    columns_duplicated = columns_from_year + other_col # Year included
    
    df_no_duplicates = sorted_df.drop_duplicates(subset=columns_from_year, keep='first')
    return df_no_duplicates.sort_values(by=['Company Name','Year'], ascending=True)

# Call udf
is_y = remove_and_sort_duplicates(income_statement_annual_df1)
bs_y = remove_and_sort_duplicates(balance_sheet_annual_df1)
cf_y = remove_and_sort_duplicates(cash_flow_annual_df1)
rt_y = remove_and_sort_duplicates(ratio_table_annual_df)

**<u>Check if duplicated values has been removed</u>**

In [28]:
income_statement_duplicates = get_duplicates(is_y)
balance_sheet_annual_duplicates = get_duplicates(bs_y)
cash_flow_annual_duplicates = get_duplicates(cf_y)
ratio_table_duplicates = get_duplicates(rt_y)

In [29]:
len(ratio_table_duplicates['Company Name'].unique())

0

**<u>Rechange 'Year' data type**

In [30]:
# -- 'category' type will help up in future models
def convert_to_category(df, last_col='Year'):
    
    # Get columns list
    cols = df.columns.tolist()

    # Get speicified column index
    index = cols.index(last_col)

    # Select columns till column specified
    cols_to_convert = cols[:index+1]

    # Convert previous columns to 'category'
    for col in cols_to_convert:
        df[col] = df[col].astype('category')
    
    return df

In [31]:
balance_sheet_annual = convert_to_category(bs_y)
income_statement_annual = convert_to_category(is_y)
cash_flow_annual = convert_to_category(cf_y)
ratio_table_annual = convert_to_category(rt_y)

## Results example of each statement table

**Balance Sheet**

In [32]:
len(balance_sheet_annual)

1521

In [33]:
balance_sheet_annual.head(5)

,Sector,Super Sector,Industry,Country,Exchange,Ticker,Company Name,Year,Total Assets (USD),Total Debt (USD),Total Equity (USD),Total Liabilities (USD),Total Shareholders Equity (USD)
232,Consumer Goods,Food Products,Food/Beverages/Tobacco,Sweden,XSTO,AAK,AAK AB,2019,2452734.83,497823.84,1115074.04,1337660.79,1106513.01
233,Consumer Goods,Food Products,Food/Beverages/Tobacco,Sweden,XSTO,AAK,AAK AB,2020,2707399.17,534310.33,1183575.76,1523823.41,1178715.08
234,Consumer Goods,Food Products,Food/Beverages/Tobacco,Sweden,XSTO,AAK,AAK AB,2021,3003120.17,506009.72,1307578.64,1695541.53,1302052.10
235,Consumer Goods,Food Products,Food/Beverages/Tobacco,Sweden,XSTO,AAK,AAK AB,2022,3262611.42,697060.43,1447489.86,1815121.57,1443650.36
656,Consumer Goods,Food Products,Food/Beverages/Tobacco,Germany,XFRA,OEL,ADM Hamburg AG,2019,207236.94,0.00,59785.06,146835.77,58351.20


**Income Statement**

In [34]:
len(income_statement_annual)

1527

In [35]:
income_statement_annual.head(5)

,Sector,Super Sector,Industry,Country,Exchange,Ticker,Company Name,Year,Revenue (USD),EBIT (USD),Interest Expense (USD),Pretax Income (USD),Net Income (USD),Net Income Available to Common (USD)
232,Consumer Goods,Food Products,Food/Beverages/Tobacco,Sweden,XSTO,AAK,AAK AB,2019,3050936.73,212955.60,16694.01,214025.73,159449.17,159449.17
233,Consumer Goods,Food Products,Food/Beverages/Tobacco,Sweden,XSTO,AAK,AAK AB,2020,3393970.33,253970.57,16404.80,246679.55,189566.55,189566.55
234,Consumer Goods,Food Products,Food/Beverages/Tobacco,Sweden,XSTO,AAK,AAK AB,2021,3918314.69,273010.92,13374.22,218850.86,159164.26,159164.26
235,Consumer Goods,Food Products,Food/Beverages/Tobacco,Sweden,XSTO,AAK,AAK AB,2022,4840644.13,248607.34,22173.09,222690.75,169897.68,169897.68
656,Consumer Goods,Food Products,Food/Beverages/Tobacco,Germany,XFRA,OEL,ADM Hamburg AG,2019,327187.92,13204.99,791.98,10283.43,8009.43,8009.43


**Cash Flow**

In [36]:
len(cash_flow_annual)

1481

In [37]:
cash_flow_annual.head(5)

,Sector,Super Sector,Industry,Country,Exchange,Ticker,Company Name,Year,Funds from Operations (USD),Changes in Working Capital (USD),Net Operating Cash Flow (USD),Capital Expenditures (USD),Net Investing Cash Flow (USD),Cash Dividends Paid Total (USD),Net Financing Cash Flow (USD),Free Cash Flow (USD)
224,Consumer Goods,Food Products,Food/Beverages/Tobacco,Sweden,XSTO,AAK,AAK AB,2019,262181.52,-95883.53,166940.07,-85717.30,-143397.24,-50189.03,NaN,82506.92
225,Consumer Goods,Food Products,Food/Beverages/Tobacco,Sweden,XSTO,AAK,AAK AB,2020,240603.70,5346.75,245464.38,-90165.63,-139744.57,-65133.12,-69264.70,159187.29
226,Consumer Goods,Food Products,Food/Beverages/Tobacco,Sweden,XSTO,AAK,AAK AB,2021,294011.77,-237641.09,56149.62,-68418.53,-71844.98,-65213.14,-7184.50,-11605.73
227,Consumer Goods,Food Products,Food/Beverages/Tobacco,Sweden,XSTO,AAK,AAK AB,2022,253406.71,-260125.83,-7007.08,-118064.49,-118064.49,-62007.85,171817.43,-123823.73
634,Consumer Goods,Food Products,Food/Beverages/Tobacco,Germany,XFRA,OEL,ADM Hamburg AG,2019,23669.82,-5791.43,17878.39,-12904.70,-12882.30,NaN,-5007.29,4984.89


**Ratio Table**

In [38]:
len(ratio_table_annual)

1497

In [39]:
ratio_table_annual.head(5)

Indices,Sector,Super Sector,Industry,Country,Exchange,Ticker,Company Name,Currency Sign,Year,Liabilities to Assets,Debt to Liabilities,Debt to Equity,InterestExp to EBIT,Return On Equity,Total_Shareholders_Equity,Operating Margin,Capex to EBIT,D&A aprox to EBITDA
220,Consumer Goods,Food Products,Food/Beverages/Tobacco,Sweden,XSTO,AAK,AAK AB,kr,2019,0.55,0.37,0.45,0.08,0.14,1.00,0.07,0.40,0.33
221,Consumer Goods,Food Products,Food/Beverages/Tobacco,Sweden,XSTO,AAK,AAK AB,kr,2020,0.56,0.35,0.45,0.06,0.16,1.00,0.07,0.36,0.17
222,Consumer Goods,Food Products,Food/Beverages/Tobacco,Sweden,XSTO,AAK,AAK AB,kr,2021,0.56,0.30,0.39,0.05,0.12,1.00,0.07,0.25,0.33
223,Consumer Goods,Food Products,Food/Beverages/Tobacco,Sweden,XSTO,AAK,AAK AB,kr,2022,0.56,0.38,0.48,0.09,0.12,1.00,0.05,0.47,0.25
620,Consumer Goods,Food Products,Food/Beverages/Tobacco,Germany,XFRA,OEL,ADM Hamburg AG,€,2019,0.71,0.00,0.00,0.06,0.14,1.00,0.04,0.98,0.54


## Optional saving (Excel)

In [88]:
income_statement_annual.to_excel("income_statement_annual.xlsx")
balance_sheet_annual.to_excel("balance_sheet_annual.xlsx")
cash_flow_annual.to_excel("cash_flow_annual.xlsx")
ratio_table_annual.to_excel("ratio_table_annual.xlsx")

time: 3.69 s (started: 2023-09-07 23:28:57 +02:00)
